<a href="https://colab.research.google.com/github/jaobigbang/jaobigbang/blob/main/process_image_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
df = pd.read_csv('data_0-151_th.csv')
df

,nameThai,number
0,เครื่องช่วยหายใจสําหรับดูแลขั้นวิกฤตในผู้ใหญ่,0
1,มาสค์สําหรับทําให้อากาศบริสุทธิ์,1
2,เครื่องช่วยหายใจที่ทําให้อากาศบริสุทธิ์,2
3,เครื่องผสมอากาศกับออกซิเจน,3
4,หน่วยระงับความเจ็บปวดที่ใช้ในการสูดดม,4
...,...,...
147,เครืองบันทึกเสียงที่เกิดจากการสั่นของหัวใจ,147
148,ตัวพิเคราะห์รังสีเบต้าและแกรมมา,148
149,เครื่องรังสีวินิจฉัยทางทันตกรรม,149
150,ยาสีฟัน,150


In [ ]:
# Create a new instance of the Chrome driver
# Set up the Chrome webdriver
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

keyword = []
for _, row in df.iterrows():
    name = row["nameThai"]
    number = row["number"]
    keyword.append((name, number))

data = []
for keyword_tuple in keyword:
    name = keyword_tuple[0]
    number = keyword_tuple[1]
    url = f'https://www.google.com/search?q={name}&tbm=isch'
    driver.get(url)
    # Scroll down to the end of the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait for the page to load
        time.sleep(1)
        # Calculate the new scroll height and check if the page has reached the end
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    images = driver.find_elements(By.CLASS_NAME, "Q4LuWd")

    # Click each image and get the large URL and thumbnail URL
    for i in images:
        thumbnail_url = i.get_attribute("src")
        title = i.get_attribute("alt")
        destination_url = ""
        large_url = ""
        try:
            i.click()
            time.sleep(1)
        except:
            continue
        try:
            destination_element = driver.find_element(By.XPATH, '/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div[2]/div/div[2]/div[2]/div[2]/c-wiz/div/div/div/div[4]/div[1]/a[1]')
            destination_url = destination_element.get_attribute("href")
        except:
            pass
        try:
            large_url_element = driver.find_element(By.XPATH, "//img[@class='r48jcc pT0Scc iPVvYb']")
            large_url = large_url_element.get_attribute("src")
        except:
            pass

        data.append({
            "name": name,
            "title": title,
            "realURL": large_url,
            "thumbnailURL": thumbnail_url,
            "destinationURL": destination_url,
            "Order": number,
            "url": url

        })

# Close the browser
driver.quit()

# Create a DataFrame from the data
df_result = pd.DataFrame(data)

# Display the DataFrame
df_result

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.133)
Stacktrace:
0   chromedriver                        0x00000001011776b8 chromedriver + 4937400
1   chromedriver                        0x000000010116eb73 chromedriver + 4901747
2   chromedriver                        0x0000000100d2c616 chromedriver + 435734
3   chromedriver                        0x0000000100d12bc1 chromedriver + 330689
4   chromedriver                        0x0000000100d12923 chromedriver + 330019
5   chromedriver                        0x0000000100d2e262 chromedriver + 442978
6   chromedriver                        0x0000000100dae56e chromedriver + 968046
7   chromedriver                        0x0000000100d93de3 chromedriver + 859619
8   chromedriver                        0x0000000100d61d7f chromedriver + 654719
9   chromedriver                        0x0000000100d630de chromed

In [ ]:
df_result.to_csv("df_data_0-151_th.csv",index=False)